In [ ]:
import numpy as np
import pandas as pd
# from tkinter import filedialog as fd
# from rdkit import Chem, DataStructs, RDLogger
# from rdkit.Chem.Draw import IPythonConsole
# from rdkit.Chem import Draw, Descriptors, AllChem
# from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator

# IPythonConsole.ipython_useSVG=True

# import numpy as np
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import accuracy_score, cohen_kappa_score, matthews_corrcoef
# import joblib


import MachineLearning as ML

In [ ]:
smiles = pd.read_csv('tested_molecules-1.csv')
smiles = smiles['SMILES']

In [ ]:
allAHDL1inhibitors = ML.importFiles(nrFiles=2)
ML.writeNewMolfile(allAHDL1inhibitors)


In [ ]:
allDescrs = ML.createDescriptorDf()
# allDescrs = allDescrs.drop(columns=["SMILES"])
x = ML.generateMorganFingerprint()
y = allAHDL1inhibitors[1][1:].astype(int)
sum(y)/len(y)
x, xDims = ML.CombineDescriptorsAndFigerprints(x,allDescrs)


In [ ]:
xScale = ML.scaleData(x,scaletype='standardize')
xPrep, _ = ML.PCAfeatureReduction(xScale, 0.999)

xTrain, xTest, yTrain, yTest, cv = ML.splitTrainTestData(xPrep,y,
                                                         printSplit=True)

In [ ]:
RFmodel, bestParam, RFgrid, param_grid, BestScoreRFgrid = ML.trainRF(xTrain, yTrain, cv)



In [ ]:
predRF, balAccRF, pred_probRF = ML.testTrainedModel(xTest, yTest, model=RFmodel, 
                                              savedModelfilename=None, scaledData=True, 
                                              scaledDatafile=None)
print(predRF,'\n', balAccRF,'\n', pred_probRF)
ML.saveTrainedModel(RFmodel, 'RandomForestClassifier_PCA80_std') # optional

In [ ]:
SVCmodel, bestParam, RFgrid, param_grid, BestScoreRFgrid = ML.trainSVC(xTrain, yTrain, cv)

In [ ]:
pred, balAcc, pred_prob = ML.testTrainedModel(xTest, 
                                              yTest, 
                                              model=SVCmodel, 
                                              savedModelfilename=None, 
                                              scaledData=True, 
                                              scaledDatafile=None)

print(pred,'\n', balAcc,'\n', pred_prob)
ML.saveTrainedModel(SVCmodel, 'SupportVectorClassifier_PCA80_std') # optional

In [ ]:
knnModel, bestParam, RFgrid, param_grid, BestScoreRFgrid = ML.trainKnn(xTrain, 
                                                                       yTrain, 
                                                                       cv, 
                                                                       list(range(1,31)))


In [ ]:
pred, balAcc, pred_prob = ML.testTrainedModel(xTest, 
                                              yTest, 
                                              model=knnModel, 
                                              savedModelfilename=None, 
                                              scaledData=True, 
                                              scaledDatafile=None)

print(pred,'\n', balAcc,'\n', pred_prob)
ML.saveTrainedModel(knnModel, 'KnnClassifier_PCA80_std') # optional

In [ ]:
daRF, threshAccRf, coverage= ML.thresholdedAccuracy(yTest, predRF, pred_probRF, threshold=0.8)
coverage

In [ ]:
threshAccRf